# Optuna

In [6]:
# Import dependencies
import numpy as np
import pandas as pd
import os, glob, subprocess, shutil, argparse
import argparse
from datetime import date

# from PIL import Image
import rasterio
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from skimage.metrics import structural_similarity as ssim

import warnings
warnings.filterwarnings("ignore")

import itertools

os.chdir('/Users/minho/Documents/GitHub/Cell2Fire')
import Cell2Fire.DataGeneratorC as DataGenerator
from Cell2Fire.Stats import *
from Cell2Fire.ParseInputs import ParseInputs
from Cell2FireC import *
from Cell2Fire.Stats import *
from Cell2Fire.Heuristics import *
from Cell2Fire.ellipse_sim import *
from Cell2Fire.ellipse_algorithms import EllipseAlgorithm

# Optimization
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from numpy import linalg as LA

from optuna.visualization.matplotlib import plot_contour
# from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_slice

## Optuna main functions

In [2]:
## Main functions
def open_img(f):
    file = glob.glob(os.path.join(f, 'Grids', 'Grids1', '*.csv'))[0]
    grid = np.loadtxt(file, delimiter=',')
#     grid = pd.read_csv(file, delimiter=',', header=None)
    return grid
    
# Objective function to optimize
class Objective:
    def __init__(self, k1_min, k1_max, k2_min, k2_max, k3_min, k3_max, k4_min, k4_max, k5_min, k5_max, loss, prune_steps, basepath):
        # Hold this implementation specific arguments as the fields of the class.
        self.k1_min = k1_min; self.k1_max = k1_max; self.k2_min = k2_min; self.k2_max = k2_max; self.k3_min = k3_min; self.k3_max = k3_max; 
        self.k4_min = k4_min; self.k4_max = k4_max; self.k5_min = k5_min; self.k5_max = k5_max;

        self.loss = loss # Integer value to select loss function
        self.prune_steps = prune_steps
        self.basepath = BASEPATH # Need BASEPATH to load fire simulation paths
        
    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        K1 = trial.suggest_float("K1", self.k1_min, self.k1_max)
        K2 = trial.suggest_float("K2", self.k2_min, self.k2_max)
        K3 = trial.suggest_float("K3", self.k3_min, self.k3_max)
        K4 = trial.suggest_float("K4", self.k4_min, self.k4_max)
        K5 = trial.suggest_float("K5", self.k5_min, self.k5_max)
        # K3=0
        # K4=0
        # K5=0

        # print(K1, K2, K3, K4, K5)

        if prune_steps and prune_steps > 0:
            # print("PRUNING!!")
            for step in range(prune_steps):
                score1, score2, score3, score4, score5 = run([K1, K2, K3, K4, K5], loss, ftype, ws, weather_hours, fperiod_length, True, BASEPATH)            
                if loss==1: 
                    intermediate_score = score1
                if loss==2: 
                    intermediate_score = score2
                if loss==3: 
                    intermediate_score = score3
                if loss==4: 
                    intermediate_score = score4
                if loss==5: 
                    intermediate_score = score5

                trial.report(intermediate_score, step)
                
                # Handle pruning based on the intermediate value.
                if trial.should_prune():
                    raise optuna.TrialPruned()
        # else:
            # print("SKIP PRUNING!!")
        score1, score2, score3, score4, score5 = run([K1, K2, K3, K4, K5], loss, ftype, ws, weather_hours, fperiod_length, True, BASEPATH)            
        
        trial.set_user_attr('fro', score1)
        trial.set_user_attr('mse', score2)
        trial.set_user_attr('f1', score3)
        trial.set_user_attr('logloss', score4)
        trial.set_user_attr('ssim', score5)
        
        if loss == 1:
            return score1
        if loss == 2:
            return score2
        if loss == 3:
            return score3
        if loss == 4:
            return score4
        if loss == 5:
            return score5

# Logging from optimization
def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)
    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)
        print(
            "Trial {} finished with best value: {} and parameters: {}. ".format(
            frozen_trial.number,
            frozen_trial.value,
            frozen_trial.params,
            )
        )

# Main processing function (Filepaths, Cell2Fire Call)
def run(K_opt,loss,ftype,ws,weather_hours,fperiod_length,save_memory,BASEPATH):

    # Set parameters
    FPERIODLENGTH = fperiod_length
    HFACTOR = 1
    FFACTOR = 1
    BFACTOR = 1
    EFACTOR = 1

    LBFORMULA = 0 # FARSITE LB Equation
    ELLIPTICAL = 4 # Adjusted alexander ellipse
    adj = 2 # Adjusted alexander ellipse (For file naming)

    # Set K Opt parameters
    K1=K_opt[0]; K2=K_opt[1]; K3=K_opt[2]; K4=K_opt[3]; K5=K_opt[4]

    # Input directories
    INSTANCE = "f"+str(ftype)+"_100_ws" + str(ws) + "_CS*" + str(weather_hours) + "HR"
    InFolder = glob.glob(os.path.join(BASEPATH, 'data_v2/data_cell2fire', INSTANCE))[0]
    OutFolder = os.path.join(BASEPATH, 'results', InFolder.split('/')[-1] + "_K1" + str(K1) + "_K2" + str(K2) + "_K3" + str(K3) + "_K4" + str(K4) + "_K5" + str(K5) + "/") # Temp folder

    # Output log
    if OutFolder is not None:
        if os.path.isdir(OutFolder) is False:
            os.makedirs(OutFolder)
        LogName = os.path.join(OutFolder, "log_"+ INSTANCE +".log")

    # Create Data.csv (if needed)
    dataName = os.path.join(InFolder, "Data.csv")
    if os.path.isfile(dataName) is False:
        print("Generating Data.csv File...")
        DataGenerator.GenDataFile(InFolder)

    # Call (Bash)
    execArray = [os.path.join(BASEPATH,"Cell2FireC/Cell2Fire"),
                "--input-instance-folder", InFolder +'/',
                "--output-folder",  OutFolder +'/', 
                "--weather", "rows", 
                "--output-message",
                "--ignitions",
                "--final-grid", 
                "--ROS-CV", "0",
                "--sim-years", "1", 
                "--nsims", "1",
                "-nweathers", "1", 
                "--Fire-Period-Length", str(FPERIODLENGTH),
                "--ROS-Threshold", "0",
                "--HFactor", str(HFACTOR), "--FFactor", str(FFACTOR), "--BFactor", str(BFACTOR), "--EFactor", str(EFACTOR),
                "--seed", "123", 
                "--EllipticalOption", str(ELLIPTICAL),
                "--KFactor1", str(K1), "--KFactor2", str(K2), "--KFactor3", str(K3), "--KFactor4", str(K4), "--KFactor", str(K5),
                "--LBFormula", str(LBFORMULA),
                "--verbose"]
                # ">", "/home/minho/fires/cell2fire/results/log_"+ INSTANCE + "_Alexander_Adjusted_LB" + str(LBFORMULA) +".log"]

    # Perform the call
    with open(LogName, 'w') as output:
        proc = subprocess.Popen(execArray, stdout=output)
        proc.communicate()
    proc.wait() 


    # FARSITE burned ellipse file path     
    farsite_path = os.path.join(BASEPATH, 'data_v2/farsite', INSTANCE)
    farsite_img_path = glob.glob(farsite_path + "*.tif")[0]

    # Mask FARSITE ROS values to binary
    # farsite_img = Image.open(farsite_img_path)
    farsite_img = rasterio.open(farsite_img_path).read(1)
    farsite_img[farsite_img < 0] = np.nan # Mask
    farsite_img[farsite_img == 0] = np.nan # Mask
    farsite_img[farsite_img > 0] = 1
    farsite_img = ~np.isnan(farsite_img)
    # print("UNique: ", np.unique(farsite_img))

    # Cell2Fire burned ellipse fire path
    img = open_img(OutFolder)
    img[img < 0] = np.nan # Mask
    img[img == 0] = np.nan # Mask
    img = ~np.isnan(img)
    # print("UNique: ", np.unique(img))



    # Set loss functions and optimization directions
    # 1: Frobenius norm
    # 2: MSE
    # 3: F1-score
    # 4: Cross-entropy (log-loss)

    score1 = LA.norm(farsite_img ^ img, ord='fro')

    score2 = mean_squared_error(farsite_img.astype(int), img.astype(int))

    score3 = np.round(f1_score(
                        y_true=farsite_img.flatten(), 
                        y_pred=img.flatten(), 
                        average="binary", 
                        zero_division=0), 2)

    score4 = log_loss(farsite_img, img, eps=1e-15)


    score5 = ssim(farsite_img, img)

    if save_memory:
        # Delete temporary folder to save memory
        shutil.rmtree(OutFolder)

    return score1, score2, score3, score4, score5


In [3]:
seed_results = []

for loss in [1,2,3,4,5]:

    for SEED in range(1,4):
        np.random.seed(SEED)

        # Input parameters to define INSTANCE
        ftype = 101
        weather_hours = 5
        fperiod_length = 1
        prune_steps = 100

        # Optuna parameters
        n_trials = 10
        # loss = 1

        # K factors
        k1_min = 0; k1_max = 1; k2_min = 1; k2_max = 2; k3_min = 0;k3_max = 0; k4_min = 0; k4_max = 0;k5_min = 0; k5_max = 0

        # Define paths (BASEPATH and OUTPATH for results)
        BASEPATH = "/home/minho/fires/cell2fireML_v2"
        OUTPATH = os.path.join(BASEPATH, 'results_paper', 'check_trials'+str(n_trials)+'_prune'+str(prune_steps)+'_loss'+str(loss))
        if OUTPATH is not None:
            if os.path.isdir(OUTPATH) is False:
                os.makedirs(OUTPATH)

        # Define dict with loss functions, name, and optimization direction
        lossdict = {1:['FrobeniusNorm', 'minimize'],
                    2:['MSE', 'minimize'],
                    3:['F1', 'maximize'],
                    4:['Log-loss', 'minimize'],
                    5:['SSIM','maximize']}

        results = []

        # Create list of wind speeds to simulate (Wind speeds in mph)
        # wind_list = list(range(min_wind, max_wind+1, step_wind))
        wind_list = [0,1,2,9,10,15,20,30]
        df_list = []

        ###################
        # Optimization loop
        for ws in wind_list:
            print("Wind speed: ", ws)
            # From the benchmark results which are available at optuna/optuna - wiki “Benchmarks with Kurobako”, at least for not deep learning tasks, we would say that
            # For RandomSampler, MedianPruner is the best.
            # For TPESampler, HyperbandPruner is the best.
            # study = optuna.create_study(directions=[lossdict[loss][1]],
            #                             sampler=optuna.samplers.TPESampler(seed=SEED),
            #                             pruner=optuna.pruners.HyperbandPruner(n_warmup_steps=10))
            study = optuna.create_study(directions=[lossdict[loss][1]],
                                        sampler=optuna.samplers.CmaEsSampler(with_margin=True),
                                        # pruner=optuna.pruners.MedianPruner())
                                        pruner=optuna.pruners.HyperbandPruner())


            study.optimize(Objective(k1_min=k1_min, k1_max=k1_max, k2_min=k2_min, k2_max=k2_max, k3_min=k3_min, k3_max=k3_max, \
                                    k4_min=k4_min, k4_max=k4_max, k5_min=k5_min, k5_max=k5_max, \
                                    loss=loss, prune_steps=prune_steps,basepath=BASEPATH), \
                                    n_trials=n_trials, callbacks=[logging_callback], show_progress_bar=True, n_jobs=4)
            # study.optimize(Objective(k1_min=k1_min, k1_max=k1_max, k2_min=k2_min, k2_max=k2_max, k3_min=k3_min, k3_max=k3_max, k4_min=k4_min, k4_max=k4_max, k5_min=k5_min, k5_max=k5_max,loss=loss, prune_steps=prune_steps, basepath=BASEPATH),n_trials=n_trials, callbacks=[logging_callback])

            fdict = {"WS":ws, "Score":study.best_value}
            fdict.update(study.best_params)
            results.append(fdict)
                
            # Plot & Visualize
            plt.rcParams['figure.figsize'] = [12, 12]
            plot_parallel_coordinate(study)
            plt.savefig(os.path.join(OUTPATH, "parallel_coordinate_"+"ws"+str(ws)+"_trials"+str(n_trials)+'_seed'+str(SEED)+".png"),  bbox_inches='tight')

            plot_optimization_history(study, target_name=lossdict[loss][0], error_bar=True)
            plt.savefig(os.path.join(OUTPATH, "plot_optimization_history"+"ws"+str(ws)+"_trials"+str(n_trials)+'_seed'+str(SEED)+".png"),  bbox_inches='tight')

            plot_contour(study, target_name=lossdict[loss][0])
            plt.savefig(os.path.join(OUTPATH, "plot_contour"+"ws"+str(ws)+"_trials"+str(n_trials)+'_seed'+str(SEED)+".png"),  bbox_inches='tight')

            # plot_intermediate_values(study)
            # plt.savefig(os.path.join(OUTPATH, "plot_intermediate_values"+"ws"+str(ws)+"_trials"+str(n_trials)+".png"),  bbox_inches='tight')

            plot_param_importances(
                study, target=lambda t: t.duration.total_seconds(), target_name="duration"
            )
            plt.savefig(os.path.join(OUTPATH, "plot_param_importances"+"ws"+str(ws)+"_trials"+str(n_trials)+'_seed'+str(SEED)+".png"),  bbox_inches='tight')

            # Parameter Results DF
            df1 = study.trials_dataframe().reset_index(drop=True)
            df_list.append(df1)
            
            
        results = pd.DataFrame(results)
        results_df = pd.concat(df_list, axis=0, keys=[str(i) for i in wind_list])

        # Convert wind speed to eccentricity values
        alpha = 0.2566; beta = -0.1548; ebar_list = []
        for ws_mph in results['WS']:
            ws_ms = ws_mph / 2.237
            LW = 0.936 * np.exp(alpha * ws_ms) + 0.461 * np.exp(beta * ws_ms) - 0.397
            Ebar = np.sqrt(1 - 1/(LW**2))
            ebar_list.append(Ebar)
        results['Ecc'] = ebar_list
        results = results[['Ecc','Score','K1','K2','K3','K4','K5']]

        # Save results
        results.to_csv(os.path.join(OUTPATH, 'k_opt_'+lossdict[loss][0]+'_seed'+str(SEED)+'.csv'), index=False)

        seed_results.append(results_df)

    # Concatenate all seed results in single DF
    seeds_df = pd.concat(seed_results, keys=['s1','s2','s3','s4','s5'])
    seeds_df = seeds_df.reset_index()

    # Select best params for all wind speeds in each seed
    ws0 = seeds_df[seeds_df['level_1']=='0']
    ws1 = seeds_df[seeds_df['level_1']=='1']
    ws2 = seeds_df[seeds_df['level_1']=='2']
    ws9 = seeds_df[seeds_df['level_1']=='9']
    ws10 = seeds_df[seeds_df['level_1']=='10']
    ws15 = seeds_df[seeds_df['level_1']=='15']
    ws20 = seeds_df[seeds_df['level_1']=='20']
    ws30 = seeds_df[seeds_df['level_1']=='30']

    # Plot CI for loss function
    import seaborn as sns
    plt.figure()
    sns.lineplot(data=ws0, x="number", y="value", label='ws0')
    sns.lineplot(data=ws1, x="number", y="value", label='ws1')
    sns.lineplot(data=ws2, x="number", y="value", label='ws2')
    sns.lineplot(data=ws9, x="number", y="value",label='ws9')
    sns.lineplot(data=ws10, x="number", y="value",label='ws10')
    sns.lineplot(data=ws15, x="number", y="value",label='ws15')
    sns.lineplot(data=ws20, x="number", y="value",label='ws20')
    sns.lineplot(data=ws30, x="number", y="value",label='ws30')

    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)

    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_perWS.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws0, x="number", y="value", label='ws0')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS0.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws1, x="number", y="value", label='ws1')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS1.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws2, x="number", y="value", label='ws2')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS2.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws9, x="number", y="value",label='ws9')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS9.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws10, x="number", y="value",label='ws10')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS10.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws15, x="number", y="value",label='ws15')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS15.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws20, x="number", y="value",label='ws20')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS20.png"),  bbox_inches='tight')

    plt.figure()
    sns.lineplot(data=ws30, x="number", y="value",label='ws30')
    plt.xlabel("Trials", fontsize=14)
    plt.ylabel(lossdict[loss][0], fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=14)
    plt.savefig(os.path.join(OUTPATH, lossdict[loss][0] + "_WS30.png"),  bbox_inches='tight')

Wind speed:  0


  0%|          | 0/10 [00:00<?, ?it/s]

Trial 1 finished with best value: 1.4142135623730951 and parameters: {'K1': 0.11684791818676266, 'K2': 1.428343521627474, 'K3': 0.0, 'K4': 0.0, 'K5': 0.0}. 


[W 2023-04-28 00:59:00,229] Output figures of this Matplotlib-based `plot_contour` function would be different from those of the Plotly-based `plot_contour`.
[W 2023-04-28 00:59:00,230] Param K3 unique value length is less than 2.
[W 2023-04-28 00:59:00,231] Param K4 unique value length is less than 2.
[W 2023-04-28 00:59:00,232] Param K5 unique value length is less than 2.
[W 2023-04-28 00:59:00,233] Param K3 unique value length is less than 2.
[W 2023-04-28 00:59:00,234] Param K4 unique value length is less than 2.
[W 2023-04-28 00:59:00,235] Param K5 unique value length is less than 2.
[W 2023-04-28 00:59:00,236] Param K3 unique value length is less than 2.
[W 2023-04-28 00:59:00,237] Param K3 unique value length is less than 2.
[W 2023-04-28 00:59:00,237] Param K4 unique value length is less than 2.
[W 2023-04-28 00:59:00,238] Param K5 unique value length is less than 2.
[W 2023-04-28 00:59:00,239] Param K4 unique value length is less than 2.
[W 2023-04-28 00:59:00,239] Param K4 un

Wind speed:  1


...

  0%|          | 0/10 [00:00<?, ?it/s]

Trial 1 finished with best value: 0.0177 and parameters: {'K1': 0.21123908370725697, 'K2': 1.486816522019637, 'K3': 0.0, 'K4': 0.0, 'K5': 0.0}. 
Trial 0 finished with best value: 0.0096 and parameters: {'K1': 0.5174515259518713, 'K2': 1.1205352437952156, 'K3': 0.0, 'K4': 0.0, 'K5': 0.0}. 


[W 2023-04-28 10:55:38,190] Output figures of this Matplotlib-based `plot_contour` function would be different from those of the Plotly-based `plot_contour`.
[W 2023-04-28 10:55:38,192] Param K3 unique value length is less than 2.
[W 2023-04-28 10:55:38,192] Param K4 unique value length is less than 2.
[W 2023-04-28 10:55:38,193] Param K5 unique value length is less than 2.
[W 2023-04-28 10:55:38,194] Param K3 unique value length is less than 2.
[W 2023-04-28 10:55:38,194] Param K4 unique value length is less than 2.
[W 2023-04-28 10:55:38,195] Param K5 unique value length is less than 2.
[W 2023-04-28 10:55:38,196] Param K3 unique value length is less than 2.
[W 2023-04-28 10:55:38,196] Param K3 unique value length is less than 2.
[W 2023-04-28 10:55:38,197] Param K4 unique value length is less than 2.
[W 2023-04-28 10:55:38,197] Param K5 unique value length is less than 2.
[W 2023-04-28 10:55:38,198] Param K4 unique value length is less than 2.
[W 2023-04-28 10:55:38,199] Param K4 un

Wind speed:  15


  0%|          | 0/10 [00:00<?, ?it/s]

In [128]:
# results_df = pd.concat(df_list, axis=0, keys=[str(i) for i in wind_list])
# results_df=results_df.reset_index()
# results_df=results_df.drop(['level_1'],axis=1)
# # results_df=results_df.set_index(['level_0'])

# ws9 = results_df[results_df['level_0']=='9']
